# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [1]:
DEVICE_SOURCE = "Zien_device1"

TIME_WINDOW = "30s"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [3]:
sys_data_file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Zien_device1/merged_data.npz')
assert sys_data_file_path.exists(), 'Data not found'
vocab_file = Path("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Zien_device1/vocabulary.pkl")
assert vocab_file.exists(), f"Vocab file at {vocab_file} does not exist."

In [4]:
import sys
repo_base_path = Path("./../../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [5]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [6]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
csv    530
zip     92
Name: filetype, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
FLS_data        91
KERN_data       91
RES_data        91
block_data      91
network_data    91
entropy_data    75
dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [7]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(60, 9)


### Get First TimeStamp

In [8]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

In [9]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-10-28 17:01:46 and 2023-11-18 08:57:39'

### Resample files

In [10]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [11]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

/tmp/ipykernel_45522/1828661645.py:3: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for source, group_df in df.groupby(["data_source"]):


Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (480, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
Data type: DataSource.FileSystem
Shape: (481, 54)
same_df shape: (4325, 57)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (482, 56)
Data type: DataSource.KernelEvents
Shape: (482, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
Data type: DataSource.KernelEvents
Shape: (482, 56)
Data type: DataSource.KernelEvents
Shape: (481, 56)
same_df shape: (4332, 59)
Data type: DataSource.Resource

5

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [12]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

In [13]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['block:block_bio_backmerge', 'writeback:global_dirty_state', 'seconds', 'jbd2:jbd2_handle_stats', 'writeback:writeback_dirty_inode', 'block:block_getrq', 'ext4:ext4_unlink_enter', 'jbd2:jbd2_start_commit', 'block:block_unplug', 'ext4:ext4_free_inode', 'writeback:sb_clear_inode_writeback', 'writeback:writeback_mark_inode_dirty', 'ext4:ext4_writepages_result', 'writeback:writeback_dirty_page', 'jbd2:jbd2_checkpoint_stats', 'ext4:ext4_da_update_reserve_space', 'block:block_touch_buffer', 'ext4:ext4_releasepage', 'block:block_dirty_buffer', 'filemap:mm_filemap_delete_from_page_cache', 'block:block_bio_remap', 'filemap:mm_filemap_add_to_page_cache', 'writeback:writeback_dirty_inode_enqueue', 'ext4:ext4_es_lookup_extent_enter', 'ext4:ext4_journal_start', 'jbd2:jbd2_handle_start', 'label', 'device', 'data_source', 'timer:timer_start', 'kmem:mm_page_free', 'kmem:kmem_cache_alloc', 'kmem:kfree', 'task:task_newtask', 'page-faults', 'sched:sched_process_exec', 'cpu-migrations', 'tcp:

In [14]:
df_all = pd.concat(df_mean_collection[:-1], axis=1).merge(df_mean_collection[-1], how='outer', left_index=True, right_index=True)
df_all.sample(2)

,block:block_bio_backmerge_FLS_data,writeback:global_dirty_state_FLS_data,jbd2:jbd2_drop_transaction,seconds_FLS_data,jbd2:jbd2_handle_stats_FLS_data,writeback:writeback_dirty_inode_FLS_data,ext4:ext4_truncate_exit,ext4:ext4_es_lookup_extent_exit,block:block_getrq_FLS_data,jbd2:jbd2_commit_flushing,...,bus-cycles,timer:timer_init,branch-misses,branch-loads,kmem:mm_page_alloc_zone_locked_RES_data,util,write_ops,write_kbs,(OLD) AverageLength,(OLD) VarianceLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-18 06:16:46,15.0,1.0,3.0,5.066072,1265.0,1690.0,9.0,698.0,11.0,1.0,...,9.004647e+09,1195.0,116163380.0,2.025735e+09,6254.0,7.526667,25.066667,2.900000,761.597701,491842.033948
2023-11-09 15:36:46,0.0,1.0,0.0,5.057323,12.0,1220.0,2.0,402.0,2.0,0.0,...,8.687285e+09,1053.0,127552394.0,1.831118e+09,7549.5,10.556667,3.066667,0.466667,1201.506908,323005.133197


In [15]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [16]:
df_all.shape, len(SELECTED_FEATURES)

((21649, 189), 260)

In [17]:
df_all

,block:block_bio_backmerge_FLS_data,writeback:global_dirty_state_FLS_data,jbd2:jbd2_drop_transaction,seconds_FLS_data,jbd2:jbd2_handle_stats_FLS_data,writeback:writeback_dirty_inode_FLS_data,ext4:ext4_truncate_exit,ext4:ext4_es_lookup_extent_exit,block:block_getrq_FLS_data,jbd2:jbd2_commit_flushing,...,timer:timer_init,branch-misses,branch-loads,kmem:mm_page_alloc_zone_locked_RES_data,util,write_ops,write_kbs,(OLD) AverageLength,(OLD) VarianceLength,label
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,744.500000,1.204354e+08,1.673524e+09,14397.500000,5.123333,19.906667,1.766667,788.910072,487441.097216,2_ransomware
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,744.500000,1.204354e+08,1.673524e+09,14397.500000,5.123333,19.906667,1.766667,788.910072,487441.097216,2_ransomware
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,744.500000,1.204354e+08,1.673524e+09,14397.500000,5.123333,19.906667,1.766667,788.910072,487441.097216,2_ransomware
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,744.500000,1.204354e+08,1.673524e+09,14397.500000,5.123333,19.906667,1.766667,788.910072,487441.097216,2_ransomware
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,744.500000,1.204354e+08,1.673524e+09,14397.500000,5.123333,19.906667,1.766667,788.910072,487441.097216,2_ransomware
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-18 08:56:46,14.5,2.0,1.5,5.135331,1380.0,1775.0,3.5,720.0,11.5,1.0,...,1181.333333,1.109333e+08,1.451889e+09,5436.333333,6.990000,25.800000,2.750000,1145.504621,364186.224516,8_bdvl
2023-11-18 08:57:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1166.000000,1.262115e+08,1.933297e+09,5663.000000,8.636667,19.066667,2.500000,958.304487,462834.315350,8_bdvl
2023-11-18 08:57:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1166.000000,1.262115e+08,1.933297e+09,5663.000000,8.636667,19.066667,2.500000,958.304487,462834.315350,8_bdvl
2023-11-18 08:57:16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1166.000000,1.262115e+08,1.933297e+09,5663.000000,8.636667,19.066667,2.500000,958.304487,462834.315350,8_bdvl


# Add Sys Call Data

In [18]:
def read_in_preprocessed_Xi_sensor_2():
    with np.load(sys_data_file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer(vocab_file=vocab_file)
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Xi_sensor_2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(11068, 154)
Loaded set: ('accept', 'accept4', 'access', 'add_key', 'adjtimex', 'bind', 'brk', 'capget', 'capset', 'chdir', 'chmod', 'chown32', 'clock_gettime', 'clone', 'close', 'connect', 'dup', 'dup2', 'dup3', 'epoll_ctl', 'epoll_wait', 'eventfd2', 'execve', 'exit', 'exit_group', 'faccessat', 'fadvise64_64', 'fallocate', 'fchdir', 'fchmod', 'fchmodat', 'fchown32', 'fcntl64', 'fgetxattr', 'flistxattr', 'flock', 'fsetxattr', 'fstat64', 'fstatat64', 'fstatfs64', 'fsync', 'ftruncate64', 'futex', 'getcwd', 'getdents64', 'getegid32', 'geteuid32', 'getgid32', 'getgroups32', 'getpeername', 'getpgid', 'getpgrp', 'getpid', 'getppid', 'getrandom', 'getresgid32', 'getresuid32', 'getsid', 'getsockname', 'getsockopt', 'gettid', 'getuid32', 'getxattr', 'inotify_add_watch', 'inotify_rm_watch', 'ioctl', 'ioprio_set', 'kcmp', 'keyctl', 'kill', 'lchown32', 'lgetxattr', 'llseek', 'lseek', 'lstat64', 'madvise', 'mkdir', 'mmap2', 'mount', 'mprotect', 'mremap', 'msync', 'munmap', 'name_to_handle

In [19]:
sys_df.head(2)

,accept,accept4,access,add_key,adjtimex,bind,brk,capget,capset,chdir,...,uname,unlink,unlinkat,utimensat,utimes,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-28 17:01:46,0.0,0.000061,0.014508,0.0,0.00000,0.000107,0.019331,0.000293,0.000082,0.000081,...,0.004723,0.000531,0.0,0.0,0.0,0.008945,0.000184,0.014028,0.000060,2_ransomware
2023-10-28 17:01:57,0.0,0.000157,0.019665,0.0,0.00104,0.000000,0.017745,0.000284,0.000080,0.000052,...,0.004887,0.000738,0.0,0.0,0.0,0.012140,0.000112,0.030913,0.000058,2_ransomware


In [21]:
if sys_df["label"].unique().shape[0] != 9:
    raise Exception(f'Not all labels!!!: {sys_df["label"].unique()}')
sys_df["label"].unique()

array(['2_ransomware', '9_xmrig', '5_httpbackdoor', '4_bashlite',
       '3_thetick', '6_beurk', '7_backdoor', '1_normal', '8_bdvl'],
      dtype=object)

In [22]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [23]:
sys_df_mean.head(2)

,faccessat,send,setsockopt,recv,prctl,mremap,recvfrom,uname,rmdir,open,...,pipe,readlink,connect,rt_sigaction,geteuid32,rename,rt_sigprocmask,llseek,brk,setresuid32
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-10-28 17:01:46,0.001078,0.005153,0.005249,0.005755,0.000208,0.000024,0.000115,0.004687,0.000238,0.0,...,0.004159,0.002193,0.003421,0.120694,0.012238,0.000040,0.052666,0.462293,0.017783,0.000158
2023-10-28 17:02:16,0.001144,0.010367,0.008288,0.010860,0.000243,0.000024,0.000138,0.004676,0.000216,0.0,...,0.004449,0.002291,0.006167,0.127959,0.013621,0.000047,0.054602,0.484687,0.017186,0.000152


In [24]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (76826, 288)


In [25]:
all_df1.head(4)

,block:block_bio_backmerge_FLS_data,writeback:global_dirty_state_FLS_data,jbd2:jbd2_drop_transaction,seconds_FLS_data,jbd2:jbd2_handle_stats_FLS_data,writeback:writeback_dirty_inode_FLS_data,ext4:ext4_truncate_exit,ext4:ext4_es_lookup_extent_exit,block:block_getrq_FLS_data,jbd2:jbd2_commit_flushing,...,pipe,readlink,connect,rt_sigaction,geteuid32,rename,rt_sigprocmask,llseek,brk,setresuid32
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,0.004159,0.002193,0.003421,0.120694,0.012238,0.00004,0.052666,0.462293,0.017783,0.000158
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,0.004159,0.002193,0.003421,0.120694,0.012238,0.00004,0.052666,0.462293,0.017783,0.000158
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,0.004159,0.002193,0.003421,0.120694,0.012238,0.00004,0.052666,0.462293,0.017783,0.000158
2023-10-28 17:01:46,8.5,1.5,2.5,5.104378,118.0,612.0,9.0,768.0,13.5,1.5,...,0.004159,0.002193,0.003421,0.120694,0.012238,0.00004,0.052666,0.462293,0.017783,0.000158


In [26]:
spath = repo_base_path / "training" / "data" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))